## Notebook to generate collision rate violators for any RMG job while running

In [1]:
import sys, os
sys.path.append('/Users/ebubeajulu/anaconda3/envs/rmg_py3/')

!conda list rmg_py3
!which python

# packages in environment at /Users/ebubeajulu/anaconda3/envs/rmg_py3:
#
# Name                    Version                   Build  Channel
/usr/bin/python


In [2]:
import rmgpy
import unittest



import numpy as np

from rmgpy.rmgobject import RMGObject, expand_to_dict, recursive_make_object


In [3]:
parent_dir = "/Users/ebubeajulu/Code/GTL/nc10/no_lowT/"

directory = "chemkin"

path = os.path.join(parent_dir, directory)

try:
    os.makedirs(path, exist_ok=True)
    print("Directory '%s' created successfully" %directory) 
except OSError as error:
    print("Directory '%s' can not be created")



Directory 'chemkin' created successfully


In [4]:
#Open chemkin file and species dictionary 

chemkinFile = open('chemkin/chem.inp','w'), open('chemkin/species_dictionary.txt', 'w')


In [5]:
import rmgpy 
from rmgpy.chemkin import load_chemkin_file
rmgpy.chemkin.load_chemkin_file('chemkin/chem.inp','chemkin/species_dictionary.txt')


ModuleNotFoundError: No module named 'quantities'

In [6]:
def check_collision_limit_violation(self, t_min, t_max, p_min, p_max):
        """
        Warn if a core reaction violates the collision limit rate in either the forward or reverse direction
        at the relevant extreme T/P conditions. Assuming a monotonic behaviour of the kinetics.
        Returns a list with the reaction object and the direction in which the violation was detected.
        """
        conditions = [[t_min, p_min]]
        if t_min != t_max:
            conditions.append([t_max, p_min])
        if self.kinetics.is_pressure_dependent() and p_max != p_min:
            conditions.append([t_min, p_max])
            if t_min != t_max:
                conditions.append([t_max, p_max])
        logging.debug("Checking whether reaction {0} violates the collision rate limit...".format(self))
        violator_list = []
        kf_list = []
        kr_list = []
        collision_limit_f = []
        collision_limit_r = []
        for condition in conditions:
            if len(self.reactants) >= 2:
                try:
                    collision_limit_f.append(self.calculate_coll_limit(temp=condition[0], reverse=False))
                except ValueError:
                    continue
                else:
                    kf_list.append(self.get_rate_coefficient(condition[0], condition[1]))
            if len(self.products) >= 2:
                try:
                    collision_limit_r.append(self.calculate_coll_limit(temp=condition[0], reverse=True))
                except ValueError:
                    continue
                else:
                    kr_list.append(self.generate_reverse_rate_coefficient().get_rate_coefficient(condition[0], condition[1]))
        if len(self.reactants) >= 2:
            for i, k in enumerate(kf_list):
                if k > collision_limit_f[i]:
                    ratio = k / collision_limit_f[i]
                    condition = '{0} K, {1:.1f} bar'.format(conditions[i][0], conditions[i][1] / 1e5)
                    violator_list.append([self, 'forward', ratio, condition])
        if len(self.products) >= 2:
            for i, k in enumerate(kr_list):
                if k > collision_limit_r[i]:
                    ratio = k / collision_limit_r[i]
                    condition = '{0} K, {1:.1f} bar'.format(conditions[i][0], conditions[i][1] / 1e5)
                    violator_list.append([self, 'reverse', ratio, condition])
        return violator_list